# Deducción de las ecuaciones diferenciales parciales de Cauchy-Navier en coordenadas cilíndricas

A continuación se presentarán dos verificaciones independientes de las ecuaciones de Cauchy-Navier en coordenadas cilíndricas:

In [1]:
from sympy import sin, cos, diff, Function, simplify, symbols, factor, Eq, init_printing, trigsimp, expand, latex, Matrix

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
r, t, z, x, y, lamda, G = symbols("r, theta, z, x, y, lamda, G")

# Se definen los desplazamientos en coordenadas cilíndricas
ur = Function("u_r")(r, t, z)
vt = Function("v_θ")(r, t, z)
w  = Function("w")(r, t, z)

# Se definen las fuerzas másicas en coordenadas cilíndricas
br = Function("b_r")(r, t, z)
bt = Function("b_θ")(r, t, z)
bz = Function("b_z")(r, t, z)

In [3]:
# Matriz de transformación de la base {r,t,z} a la base {i,j,k}
T = Matrix([[ cos(t), -sin(t),       0 ],     
            [ sin(t),  cos(t),       0 ],      
            [      0,       0,       1 ]])

Se definen los desplazamientos en coordenadas rectangulares y cilíndricas:

In [4]:
# vector de desplazamientos con respecto a la base {r,t,z}
u_cil = Matrix([ ur,
                 vt,
                 w   ])

# vector de desplazamientos con respecto a la base {i,j,k}
u_rec = T * u_cil

# componentes de los desplazamientos en coordenadas rectangulares
u, v = u_rec[0], u_rec[1]

Se definen las fuerzas másicas en coordenadas rectangulares y cilíndricas:

In [5]:
# vector de fuerzas másicas con respecto a la base {r,t,z}
b_cil = Matrix([ br,     
                 bt,      
                 bz  ])

# vector de fuerzas másicas con respecto a la base {i,j,k}
b_rec = T * b_cil

# componentes de la fuerza másica en coordenadas rectangulares
X, Y, Z = b_rec

In [6]:
# se definen las primeras derivadas
d_dx   = lambda f: diff(f,r)*cos(t) - diff(f,t)*sin(t)/r
d_dy   = lambda f: diff(f,r)*sin(t) + diff(f,t)*cos(t)/r
d_dz   = lambda f: diff(f,z)

# se definen las segundas derivadas
d2_dx2 = lambda f: d_dx(d_dx(f))
d2_dy2 = lambda f: d_dy(d_dy(f))
d2_dz2 = lambda f: d_dz(d_dz(f))

Se define la divergencia de un campo vectorial en coordenadas rectangulares y cilíndricas:

In [7]:
# divergencia de un campo vectorial en coordenadas rectangulares y cilíndricas
div_rec = lambda v: d_dx(v[0]) + d_dy(v[1]) + d_dz(v[2])

div_cil = lambda v: diff(v[0],r) + v[0]/r + (1/r)*diff(v[1],t) + diff(v[2],z)

Se define el laplaciano de una función en coordenadas rectangulares y cilíndricas:

In [8]:
# laplaciano en coordenadas rectangulares y cilíndricas
lapl_rec = lambda f : simplify(d2_dx2(f) + d2_dy2(f) + d2_dz2(f))

lapl_cil = lambda f : simplify((1/r)*diff(r*diff(f,r),r) + (1/(r**2))*diff(f,t,2) + diff(f,z,2))

Se define el laplaciano vectorial en coordenadas rectangulares y cilíndricas:

In [9]:
# laplaciano de un campo vectorial rectangular (laplaciano vectorial) en coordenadas rectangulares y cilíndricas
lapl_vec_rec = lambda v : Matrix([ lapl_rec(v[0]),
                                   lapl_rec(v[1]),
                                   lapl_rec(v[2]) ])

lapl_vec_cil = lambda v : Matrix([ lapl_cil(v[0]) - v[0]/(r**2) - (2/(r**2))*diff(v[1],t),
                                   lapl_cil(v[1]) - v[1]/(r**2) + (2/(r**2))*diff(v[0],t),
                                   lapl_cil(v[2])                                         ])

Se define el gradiente de una función en coordenadas rectangulares y cilíndricas:

In [10]:
# gradiente en coordenadas rectangulares y cilíndricas
grad_rec = lambda f : Matrix([ d_dx(f),
                               d_dy(f),
                               d_dz(f) ])

grad_cil = lambda f : Matrix([       diff(f,r),
                               (1/r)*diff(f,t),
                                     diff(f,z) ])

Se calcula la dilatación cúbica usando las fórmulas en coordenadas rectangulares y cilíndricas:

In [11]:
# se calcula la dilatación cúbica
e_rec = expand(div_rec(u_rec).subs({sin(t): 0, cos(t): 1}))
e_cil = expand(div_cil(u_cil).subs({sin(t): 0, cos(t): 1}))

# y se verifica que ambas expresiones sean iguales
e_rec - e_cil

## Método 1: Comparando las ecuaciones en notación extendida

Se escriben las ecuaciones de Cauchy-Navier en coordenadas rectangulares y cilíndricas:

In [12]:
# Ecuaciones de Cauchy-Navier en coordenadas rectangulares:
eqCN1_rec = [ (lamda + G)*d_dx(e_rec) + G*(lapl_rec(u)) + X,
              (lamda + G)*d_dy(e_rec) + G*(lapl_rec(v)) + Y,
              (lamda + G)*d_dz(e_rec) + G*(lapl_rec(w)) + Z ]

In [13]:
# Ecuaciones de Cauchy-Navier en coordenadas cilíndricas:
eqCN1_cil = [ (lamda + G)*diff(e_cil,r)   + G*(lapl_cil(ur) - (ur/(r*r)) - (2/(r*r))*diff(vt, t)) + br,
              (lamda + G)*diff(e_cil,t)/r + G*(lapl_cil(vt) - (vt/(r*r)) + (2/(r*r))*diff(ur, t)) + bt,
              (lamda + G)*diff(e_cil,z)   + G*lapl_cil(w) + bz ]

In [14]:
eqCN1_rec = Matrix(eqCN1_rec).subs({sin(t): 0, cos(t): 1})
eqCN1_cil = Matrix(eqCN1_cil).subs({sin(t): 0, cos(t): 1})

Se verifica que ambos conjuntos de ecuaciones sean equivalentes:

In [15]:
simplify(eqCN1_rec - eqCN1_cil)

## Método 2: Comparando las ecuaciones en notación vectorial compacta

Se escriben las ecuaciones de Cauchy-Navier en coordenadas rectangulares y cilíndricas, en su notación vectorial compacta:

In [16]:
# ecuación de Cauchy-Navier en términos de desplazamientos
eqCN2_rec = (lamda + G)*grad_rec(e_rec) + G*lapl_vec_rec(u_rec) + b_rec
eqCN2_cil = (lamda + G)*grad_cil(e_cil) + G*lapl_vec_cil(u_cil) + b_cil

In [17]:
eqCN2_rec = eqCN2_rec.subs({sin(t): 0, cos(t): 1})
eqCN2_cil = eqCN2_cil.subs({sin(t): 0, cos(t): 1})

Se verifica que ambos conjuntos de ecuaciones sean equivalentes:

In [18]:
simplify(eqCN2_rec - eqCN2_cil)

Se verifica que las ecuaciones obtenidas con ambos métodos sean equivalentes:

In [19]:
simplify(eqCN1_rec - eqCN2_rec)